In [1]:
import csv
import pandas as pd
from django.contrib.gis.geos import Point
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

df = pd.read_csv("adresses-33.csv", sep=";")




/tmp/ipykernel_223/1077711013.py:7: DtypeWarning: Columns (5,7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("adresses-33.csv", sep=";")


In [3]:
print(df.keys())

print(df['certification_commune'].value_counts())

Index(['uid_adresse', 'cle_interop', 'commune_insee', 'commune_nom',
       'commune_deleguee_insee', 'commune_deleguee_nom', 'voie_nom',
       'lieudit_complement_nom', 'numero', 'suffixe', 'position', 'x', 'y',
       'long', 'lat', 'cad_parcelles', 'source', 'date_der_maj',
       'certification_commune'],
      dtype='object')
certification_commune
1    436177
0    334148
Name: count, dtype: int64


# Création de nouveaux liens bâtiments <> adresses

In [2]:
certified_df = df[df['certification_commune'] == 1]    
certified_df.reset_index(drop=True)
print(f"Adresses certifiées : {len(certified_df)}")

Adresses certifiées : 436177


In [ ]:
new_links = set()
to_scan = 200000
c = 0
start_at = 100001
for index, row in certified_df.iterrows():
    
    c += 1
    
    if c < start_at:
        continue

    point = Point(row["long"], row["lat"], 4326)
    
    bdg = Building.objects.filter(shape__intersects=point, is_active=True).exclude(addresses_id__contains=[row['cle_interop']])
    
    if bdg:
        
        couple = (bdg[0].rnb_id, row['cle_interop'])
        new_links.add(couple)
        
   
    if c % 100 == 0:
        print('--')
        print(f"row {c}")
        print(f"{len(new_links)} new links so far")
        
        
    if c >= to_scan:
        break
        

print(f"found {len(new_links)}, scanning {to_scan} rows")

# Indice pour nouveaux bâtiments

In [5]:
position_counts = certified_df['position'].value_counts()
print(position_counts)

position
entrée                335241
logement               23229
bâtiment               14941
segment                10382
parcelle                5165
délivrance postale      2867
service technique        140
cage d’escalier           49
Name: count, dtype: int64


In [3]:
bdg_df = df[df["position"] == "bâtiment"]
print(f"Adresses certifiées de type bâtiments : {len(bdg_df)}")

Adresses certifiées de type bâtiments : 15619


In [23]:
from batid.services.closest_bdg import get_closest_from_point


to_scan = 500
wo_rnb_bdgs = 0
c = 0
for index, row in bdg_df.iterrows():

    c += 1
    
    if c % 10 == 0:
        print(c)
    
    bdgs = get_closest_from_point(row["lat"],row["long"],10)[:2]
    
    if len(bdgs) == 0:
        
        wo_rnb_bdgs += 1
        
        
        #print('----')
        # print(row)
        
        address_infos = [
            str(row['numero']),
            row['voie_nom'],
            row['commune_nom']
        ]
        
        #print(row['cle_interop'])

        #print(" ".join(address_infos))
        #print(f"{row['lat']},{row['long']}")
        
        if wo_rnb_bdgs % 10 == 0:
            print(f"so far, found {wo_rnb_bdgs} bal row without RNB bdg")

    
print(f"finally found {wo_rnb_bdgs} bal row without RNB bdg")

    

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
so far, found 10 bal row without RNB bdg
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
so far, found 20 bal row without RNB bdg
540
550
560
570
580
590
600
so far, found 30 bal row without RNB bdg
610
620
630
640
650
660
670
680
690
700
so far, found 40 bal row without RNB bdg
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
so far, found 50 bal row without RNB bdg
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
so far, found 60 bal row without RNB bdg
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
so far, found 70 bal row without RNB bdg
1620
1630
1640
1

11820
11830
11840
11850
11860
11870
11880
11890
11900
11910
11920
11930
11940
11950
11960
11970
11980
11990
12000
12010
12020
12030
12040
12050
12060
12070
12080
12090
12100
12110
12120
12130
12140
12150
12160
12170
12180
12190
12200
12210
12220
12230
12240
12250
12260
12270
12280
12290
12300
12310
12320
12330
12340
so far, found 540 bal row without RNB bdg
12350
12360
12370
12380
12390
12400
12410
12420
12430
12440
12450
12460
12470
12480
12490
12500
12510
12520
12530
12540
12550
12560
12570
12580
12590
12600
12610
12620
so far, found 550 bal row without RNB bdg
12630
12640
12650
12660
12670
12680
12690
so far, found 560 bal row without RNB bdg
12700
12710
12720
12730
12740
12750
12760
12770
12780
12790
12800
12810
12820
12830
12840
12850
12860
so far, found 570 bal row without RNB bdg
12870
12880
12890
12900
12910
12920
12930
12940
12950
12960
12970
12980
12990
13000
13010
13020
13030
13040
13050
13060
13070
13080
13090
13100
13110
13120
13130
13140
13150
13160
13170
13180
13190
1320